# Data Engineer Assessment
## UC02: TTD_DE_UC02_DQC: Add Data Quality Checks to the Data Engineering Pipeline. Address Any Data Quality Issues

## Goal
As a data engineer, I want to setup quality checks to ensure that the data that is provided to business is of good quality and fit for use.

## Summary
It is important to ensure that the data provides meets the data quality standards setup by the organization and the data is fit for use.


* __Problem Statement__ - Business has experienced data with issues such as duplicates in the data or inconsistent data types than those reported by the source.

Business has asked the Data Engineering team to implement controls that ensure that the data is fit for use and detect any issues with the pipeline if this is not the case.

* __Description__ - Lately, business is reporting that the data provide by the sources have several issues. This has caused incorrect reports to be generated and poor customer experience. Business has lost trust in the data due to the poor data quality

* __Data Issues Reported__ : Business has documented and highlighted the following issues with the data over the last 3 months.

  a. Data source systems changing the schema of the data without informing business.
  b. Numerous duplicates in the sales_rep_master data in the data resulting in higher processing time and duplicates in the Semantic asset.
  c. Vehicle VIN number columns not having the right length of fields or missing data. VIN Length - 17 Characters
  d. Model Brand not in expected brand types - BMW, MINI, Rolls Royce or strings not matching the expectated format.
  e. Data is not updated during the weekly build due to the source system not having the data available.

* __Expected Outcome__ : Your manager has asked you to perform a Proof of Concept (PoC) of the Great Expectations Quality framework and demonstrate to the team the various features of the framework.

You need to evaluate the features and functionality of the [`Great Expectations Quality framework`](https://greatexpectations.io/).

1. The Framework must support integration with the AWS Platform and must be deployed on to AWS. The PoC demonstration must be done on your laptop locally.  

2. The Framework must perform Schema Validation (including data type validation), data validation for VIN number (Length:17) and not null columns (VIN numbers). Other checks that are deemed suitable can also be added.
3. The Framework must also detect if there are duplicates in the data.
4. Provide a compliance report of the data on a sample datasets.
5. Cost of the check is a concern for the IT department. Ensure that this is taken into account when implementing the checks.

## Code Complexity
- Low / Medium

## `Diagram - Also refer PDF in folder`

![Data Quality Checks](./TTD_DE_UC02_DQC.png"Great Expectation Data Quality Checks")



## Datasets:

`File Location`: Refer to the attached `data` folder for information

* Vehicles (vehicles.csv)  at the plants (plants.csv) are built to order (orders.csv) placed - order_number
* Customer (customers.csv) provides reviews(welcome_call.csv) 60 to 80 days after the vehicles are delivered(vin).
* Sales (sales_rep.csv) representatives are linked to dealership (dealers.csv) and have dealership names




## Expected Outcomes:

1. The Great Expectations framework must be used to perform the data quality checks.
2. Atleast 5 different types of checks must be implemented on each dataset.  Explaination of why the check would be appropriate for the datast must be provided.

### Libraries or Options used
* Jupyter Notebook - Install and run locally on your laptop or device.
* Great Expectations Framework (Note: Install if required)
* PySpark, Pandas and matplot lib or similar plotting libraries




## `Acceptance Criteria`

1. Only the Great Expectations framework can be used for this exercise.
2. Implement different types of checks on the three datasets (vehicles, customers, sales_rep) data and provide your findings.
3. Explain the checks you have implemented and how it would be useful in detecting Data quality issues to the business.  Refer to the current challenges that business has highlighted.
4. Explain how the framework would be useful when data is stored in an RDBMS such as MySQL. Illustrate the workflow using **draw.io** and export the output in pdf format.  Expected Output: draw.io diagram


# Implementation

Provide all the implementation steps in the sections that follow. Ensure that you provide detailed explanations of the approach.


### Step 1: Import the libraries that you need for Great Expectations framework.

In [ ]:
# Import any relevant libraries
import re
import pandas as pd
import great_expectations as gx
from great_expectations.core.batch import BatchRequest
from great_expectations.data_context.data_context import CloudDataContext,FileDataContext
from great_expectations.datasource.fluent import pandas_datasource
from great_expectations.expectations.core import ExpectTableColumnsToMatchOrderedList, ExpectColumnValuesToMatchRegex,ExpectColumnValuesToBeOfType, ExpectColumnValuesToBeInSet, ExpectColumnValuesToNotBeNull
import os
# from great_expectations.datasource.fluent.pandas_datasource
# Import the Great Expectations library below.
# Note: You can pip install any libraries that you need into your environment.


#### List of expected dataframes to be loaded


  * Vehicles (vehicles.csv) - Provides information on the vehicles.
  * Customer (customers.csv) - Provide information about the customers
  * Sales (sales_rep.csv) Provides information about the sales_reps at the dealerships.


### Step 2: Load the data from the data folder into the data frame.

In [ ]:
## Loading Dataframe from local file system as a POC, for the prototype we then move to S3 (reuired dependencies for that: s3fs,fsspec,aiobotocore)

vehicle_typo = {"hyrdogen":"hydrogen"}
customers_df = pd.read_csv("/Users/samkelom/Desktop/samkelo/data_eng/bmw/TTD_DE_UC01_EDA/data/customers.csv",delimiter='|')
customers_df= customers_df.where(customers_df['customer_id'].str.startswith('CNI'))

sales_person_df = pd.read_csv("/Users/samkelom/Desktop/samkelo/data_eng/bmw/TTD_DE_UC01_EDA/data/sales_person.csv",delimiter='|')
vehicles_df = pd.read_csv("/Users/samkelom/Desktop/samkelo/data_eng/bmw/TTD_DE_UC01_EDA/data/vehicles.csv",delimiter='|')

vehicles_df['fuel_type'] = vehicles_df['fuel_type'].replace(vehicle_typo)



In [ ]:
# Write your code below to load the relevant data into a pandas dataframe and make it available to Great Expectations.


# Initialize the DataContext
context = gx.get_context(mode='file')


## VEHICLE DATASET
# Create an expectation suite

expectation_suite_name = "schema_and_vin_validation_suite"
suite = context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

vehicle_pandas_ge_df = gx.from_pandas(vehicles_df,expectation_suite=suite)

# Schema validation expectations
column_list=["order_number","vin","fuel_type","transmission_type","prod_plant_id","production_country","dealer_number","export_country","steering_wheel_position","cabin_door_count","engine_series","drive_type","price","currency"]

# VIN number heuristic expectation
vin_pattern = r"^[A-HJ-NPR-Z0-9]{17}$"  # VIN pattern: 17 characters, excluding I, O, Q since a VIN cannot have these letters in 

# Expectations
fuel_type_expectation = vehicle_pandas_ge_df.expect_column_values_to_be_in_set(column="fuel_type",value_set=["petrol", "diesel","hydrogen and electric","hydrogen","petrol and electric","electric"])
schema_order_expectation = vehicle_pandas_ge_df.expect_table_columns_to_match_ordered_list(column_list=column_list)
vin_expectation = vehicle_pandas_ge_df.expect_column_values_to_match_regex(column="vin", regex=vin_pattern)
vehicle_model_expectation = vehicle_pandas_ge_df.expect_column_values_to_be_in_set(column="make",value_set=["BMW", "MINI", "Rolls-Royce"],mostly=.5)
schema_type_expectation = vehicle_pandas_ge_df.expect_column_values_to_be_of_type(column='price',type_="NUMBER")
null_vin_expectation = vehicle_pandas_ge_df.expect_column_values_to_not_be_null(column="vin")

validation_results = vehicle_pandas_ge_df.validate()
print(validation_results)


## CUSTOMER DATASET


expectation_suite_name = "customer_validation_suite"
suite = context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

customer_pandas_ge_df = gx.from_pandas(customers_df,expectation_suite=suite)

# Schema validation expectations
# VIN number heuristic expectation

# Expectations
valid_gender_expectation = customer_pandas_ge_df.expect_column_values_to_be_in_set(column="sex",value_set=["female", "male"],mostly=.5)
null_cust_id_expectation = customer_pandas_ge_df.expect_column_values_to_not_be_null(column="customer_id")
customer_validation_results = customer_pandas_ge_df.validate()
print(customer_validation_results)



## SALES REP DATASET

dealer_number_pattern = r'\bDN[A-Za-z]{2}\d{5}\b'
sales_rep_number_pattern = r'\bSN[A-Za-z]{2}\d{9}\b'

expectation_suite_name = "sales_rep_validation_suite"
suite = context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

sales_rep_pandas_ge_df = gx.from_pandas(sales_person_df,expectation_suite=suite)

# Schema validation expectations
# VIN number heuristic expectation

# Expectations

dealer_number_pattern = sales_rep_pandas_ge_df.expect_column_values_to_match_regex(column="dealer_number", regex=dealer_number_pattern)
sales_rep_id_expectation = sales_rep_pandas_ge_df.expect_column_values_to_match_regex(column="sales_number", regex=sales_rep_number_pattern)
emlpoyment_type_expectation = sales_rep_pandas_ge_df.expect_column_values_to_be_in_set(column="employment_type",value_set=["permanent", "contractor"],mostly=.5)
not_null_sales_id_expectation = sales_rep_pandas_ge_df.expect_column_values_to_not_be_null(column="sales_number")
customer_validation_results = sales_rep_pandas_ge_df.validate()
print(customer_validation_results)



#### Question: Did you face any challenges when setting up the environment as well as with the dataset. How did you resolve these challenges

#### *Answer*: 
# 1 Setting up the env, Installing the Great Expectations binary on MacOS, only the library gets installed with PIP (the GX docs and Stack Overflow are not helping). 
# 2 I run into error: (Downgrading to v0.18.9 solved the issue)
> "AttributeError: 'BatchRequest' object has no attribute 'options'" when running:
 >> validator = context.get_validator()
# 3 the Great Expectations Docs will take you to a 404 Page Not Found for a lot of 1.0.1 issues (realying on S/O and the dir() method to see what functions/methods are available for each class/object)




### Step 3: Explore your data and provide some column and statistical information
 Provide some statistical information about the data you just loaded


In [ ]:
# Write code to provide statistical information about each dataframe that you just loaded.

customers_df.describe()
sales_person_df.describe()
vehicles_df.describe()

### Step 4: Add your expectations Logic and for each expectation provide an explanation of why it would be useful to business.

In [ ]:
# Explore your data and add the relevant Expectations. Loop these code cells as required for each expectation.

## Vehicle dataset
# 1 I have tried to assert a regex on the source Pandas dataframe in order to validate that the VIN. The VIN number needs to adhere to vehicle industry standards and lack of this data (or invalid format) leaves behind data points that are beneficial to the business in order to understand business patterns and gain a competitive advantage
# 2 I have created an expectation to check if the 2 data frames have the same number and order of rows 
# 3 validated the Price column  to make sure it is a numeric column as we will be using it in financial reports
# 4 Made sure that the Model column does not contain data outside BMW, Mini and Rolls Royce. As we are only focusing on these 3 car makes, any data outside of them will only distract us from understanding the business production line 
# 5 Made sure fuel type is between values: ["petrol", "diesel","hydrogen and electric","hydrogen","petrol and electric","electric"]

##  Customer dataset
# 1 On the customers dataset, I made sure that the gender is set to MALE of FEMALE (as the business currently support binary genders, will include non-binary as the downstream business processes do)
# 2 Did a validation to check if the Customer ID field is indeed populated as we need to understand customer specific data 

## Sales Rep dataset
# 1 I made sure that Sales Rep ID and Dealer Number are populated in the correct format using regex 
# 2 I made sure that no other employment type is introduced other than permanent or contractor 
# 3 I enforced that Sales Number and Dealer number must not be NULL 




### Step 5: Review your expectations and document your findings.

* Note: You can check your expectations by using df.get_expectation_suite() (where df is a pandas dataframe)




In [ ]:
## 1  Vehicle Dataset: 
# While validating the VIN number, we found that with the existing dataset, the expectation is being met. 

# The Expectation enforced that the column has a value that has 17 characters and does not contain letters I, O, Q since a VIN by specification should not have these letters.
# the validation Identified that 5 out of 193705 had invalid VIN number values

# Great Expectations has validated that the fuel_types provided are all valid fuel_types so that information was captured well

# The column order seemed to have a bit of an inconsistency when it comes to column. 
# Seeing that the underlying data is .csv (which is a file format that generally gets by index, 
# we could run into downstream issues with CSV headers referencing the wrong value)


## 2  Customer Dataset:
# THe customer ID also got badly filled in  102255 out of 129038 customer ID values were NULL .i.e did not meet the 'expect_column_values_to_not_be_null' expectation
# Seeing that we are now adpoting non-binary, it looks like the sex column needs to allow more inputs that accommodate no-binary gender ()


## 3  Sales Rep Dataset:
# We had to validate the customer ID and Dealer Number values using regex 
# I also needed to verify that none of these values are NULL as they contain a lot of business value
# Also validated that valid values were being provided for the employment type

#### Question: Explain the different expectations that you implemented above and also provide your findings?

#### *Answer*: Replace with your response

### Report all your Findings:

<REPLACE TEXT BELOW>

Report your findings in bullet points.
Example: For illustration purpose only - replace below with your own findings and support with Evidence.
1. The data lenth check expectation would be useful as VIN numbers are 17 characters long. The check was implemented in the following manner by valdating that we have 17 characters and they adhere to industry specifications, we adopted regex: 

    r"^[A-HJ-NPR-Z0-9]{17}$" 
the validation Identified that 5 out of 193705 had invalid VIN number values as per my regex

2. Great Expectations has validated that the fuel_types provided are all valid fuel_types so that information was captured well so we will get accurate data in this subject

3. Great Expectations picked up some null customer IDs so we will need to work on capturing this data correctly

4. The Gender column was sparsely populated. Seeing that we are now adpoting non-binary, it looks like the sex column needs to allow more inputs that accommodate no-binary gender

5. For the SalesRep data, we confirmed that the Dealer Number and Sales Rep ID was being captured correctly and the employment type was populated with a valid type so the Sales Rep-Oriented Analytics should be quite rich as the dataset is of good quality




In [ ]:
## 4. Explain how the framework would be useful when data is stored in an RDBMS such as MySQL. Illustrate the workflow using **draw.io** and export the output in pdf format.
## Using the Great Expectations framework with Relational Databases, would be much easier to document and report from reading Relational Databases.
# When using GE with Relational Database systems, we do not need to do data type and schema validation checks like we did with the file based systems.
# This allows us to focus our attention towards the more semantic validations like the df.expect_column_values_to_be_in_set() method 



## `Acceptance Criteria`

1. Only the Great Expectations framework can be used for this exercise.
2. Implement different types of checks on the three datasets (vehicles, customers, sales_rep) data and provide your findings.
3. Explain the checks you have implemented and how it would be useful in detecting Data quality issues to the business.  Refer to the current challenges that business has highlighted.
4. Explain how the framework would be useful when data is stored in an RDBMS such as MySQL. Illustrate the workflow using **draw.io** and export the output in pdf format.